[Описание](#0)

[Подготовка данных](#1)

[Исследование таблиц](#2)

[Выполнение задач](#3)

[Выводы](#4)

# Анализ базы данных сервиса для чтения книг по подписке
<a name="0"></a>
# Описание проекта
**Цель проекта - проанализировать имеющуюся базу данных с информацией о книгах, издательствах, обзорах книг. На основании проведённого анализа сформулировать ценностное предложени для нового продукта.** 

**Цель будет достигнута путём решения следующих задач:**
1. Подсчёта книг, вышедших после 1 января 2000 года;
2. Подсчёта количества обзоров и средней оценки для каждой книги;
3. Определения издательство, выпустившее наибольшее число книг толще 50 страниц (не брошюры);
4. Определения автора с наивысшей средней оценкой книг;
5. Подсчёта среднего количества обзоров от пользователей, которые поставили больше 50 оценок.

**Располагаем данными:**
- таблица "books" с данными о книгах;
- таблица "authors" с данными об авторах;
- таблица "publishers" с данными об издательствах;
- таблица "ratings" с данными о пользовательских оценках книг;
- таблица "reviews" с данными опользовательских обзорах на книги.
***
<a name="1"></a>
## Подготовка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# выведем два числа после запятой плавающих чисел:
pd.set_option('display.float_format', '{:.2f}'.format)      

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

***
<a name="2"></a>
## Исследование таблиц
Выведем первые строки каждой таблицы

In [2]:
#Список со всеми таблицами:
tables = ['books','authors','publishers','ratings','reviews']

In [3]:
# Создадим функцию для упрощения выполнения запросов.
def show(query):
    display (pd.io.sql.read_sql(query, con = engine))

In [4]:
# изучим данные о всех таблицах:
for dataframe in tables:
    query = 'SELECT * FROM ' + dataframe + ' LIMIT 5'
    show(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [5]:
# Поищем названия книг не на английском языке
query = 'SELECT * FROM books ORDER BY book_id DESC LIMIT 5'
show(query)    

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1000,509,Zen and the Art of Motorcycle Maintenance: An ...,540,2006-04-25,143
1,999,94,You Suck (A Love Story #2),328,2007-01-16,331
2,998,201,Year of Wonders,358,2002-04-30,212
3,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
4,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147


In [6]:
# Поищем фамилии авторов не на английском языке
query = 'SELECT * FROM authors ORDER BY author DESC LIMIT 5'
show(query)    

,author_id,author
0,636,Åsne Seierstad/Ingrid Christopherson
1,635,Zora Neale Hurston
2,634,Zilpha Keatley Snyder
3,633,Zadie Smith
4,632,William Strunk Jr./E.B. White


Видим, что в базах данных используется англоязычный ввод. Общее количество книг-1000, авторов - 636. Учитывая, что у нас есть схема данных с корректно указанными типами данных, можем приступить непосредственно к выполнению задач

***
<a name="3"></a>
## Выполнение задач
### Подсчёт книг, вышедших после 1 января 2000 года

In [8]:
query = '''SELECT COUNT (*) AS cnt
           FROM books
           WHERE publication_date > '2000-01-01' '''
 
show(query)

,cnt
0,819


После 1 января было выпущено 819 книг.



### Подсчёт количества обзоров и средней оценки для каждой книги

In [9]:
query = '''
    SELECT
        books.book_id AS book_id,
        books.title AS book_title,
        COUNT (DISTINCT review_id) AS reviews_cnt,
        AVG (rating) AS avg_rating
    FROM books             -- используем LEFT JOIN для исключения потерь книг 
    LEFT JOIN reviews ON reviews.book_id = books.book_id
    LEFT JOIN ratings ON ratings.book_id = books.book_id  
    GROUP BY books.book_id   
    ORDER BY reviews_cnt DESC    
'''
show(query)

,book_id,book_title,reviews_cnt,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


Больше всего обзоров у книги "Сумерки" - 7 обзоров. Средний пользовательский рейтинг этой книги - 3,66. Что примечательно: для книг, на которых нет обзоров, рейтинг выше - "The Cat in the Hat and Other Dr. Seuss Favorites" - 5 баллов, "Essential Tales and Poems" и "Disney's Beauty and the Beast " по 4 балла. Отсюда вывод - количество отзывов не влияет на среднюю оценку.

### Определение издательства, выпустившего наибольшее число книг толще 50 страниц

In [10]:
query = '''
    SELECT
        publishers.publisher AS publisher,
        COUNT (book_id) as book_cnt
    FROM books
    INNER JOIN publishers 
        ON publishers.publisher_id = books.publisher_id    
    WHERE num_pages > 50
    GROUP BY publisher
    ORDER BY book_cnt DESC
    LIMIT 1
    '''

show(query)

,publisher,book_cnt
0,Penguin Books,42


Лидером по выпуску книг толще 50 страниц является издательство "Penguin Books", которое выпустило 42 книги.


### Определение автора с самой высокой средней оценкой книг
Для решения задания необходимо учитывать только книги с 50 и более оценками.

In [11]:
query = '''
    SELECT 
        SUBQ.author AS author,
        AVG(SUBQ.avg_rating) AS avg_rating
    FROM(
        SELECT    -- создадим подзапрос для книг с более 50-ю оценками
            authors.author AS author,
            books.book_id AS id,
            AVG (rating) AS avg_rating,
            COUNT (ratings.rating) AS rating_cnt
        FROM books         
        LEFT JOIN authors ON authors.author_id = books.author_id
        LEFT JOIN ratings ON ratings.book_id = books.book_id
        GROUP BY author, id
        HAVING COUNT(ratings.rating) >= 50
            ) AS SUBQ
    GROUP BY
        author
    ORDER BY
        avg_rating DESC
    LIMIT 1     
    '''

show(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


Автором с самой высокой средней оценкой книг - является Дж. К. Роулинг. Учитывая, что рядом указан иллюстратор обложек книг серии о "Гарри Поттере" -  Мэри Грандпре, можно сказать что именно эта серия набрала самую высокую среднюю оценку.



### Подсчёт среднего количества обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
query = '''
    SELECT
        AVG (SUBQ.review_cnt) AS avg_rev_cnt
        FROM
            (
                SELECT
                    COUNT (review_id) as review_cnt
                FROM
                    reviews
                WHERE
                    username IN
                                (-- выделим пользователей, которые поставили больше 50-ти оценок:
                                    SELECT username
                                    FROM ratings
                                    GROUP BY username
                                    HAVING COUNT (ratings) > 50
                                )
                GROUP BY username
            ) AS SUBQ
    '''

show(query)

,avg_rev_cnt
0,24.33


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24 обзора. 


***
<a name="4"></a>
## Выводы и итоги
- В базах данных используется англоязычный ввод;
- После 1 января было выпущено 819 книг, 81,9% от представленной базы;
- Составлена таблица с количество обзоров и средней оценкой для каждой книги. Больше всего обзоров у книги "Сумерки" - 7 обзоров (но средний бал - 3,66);
- Лидером по выпуску книг толще 50 страниц является издательство "Penguin Books", которое выпустило 42 книги;
- Автором с самой высокой средней оценкой книг - является Дж. К. Роулинг;
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24 обзора.